In [ ]:
import sys
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
my_local_drive='/content/gdrive/My\ Drive/TER/WhaleFaceNet/'
# Ajout du path pour les librairies, fonctions et données
sys.path.append(my_local_drive)
# Se positionner sur le répertoire associé
%cd $my_local_drive

%pwd

/content/gdrive/My Drive/TER/WhaleFaceNet


'/content/gdrive/My Drive/TER/WhaleFaceNet'

In [ ]:
import os
import numpy as np
import tensorflow.keras.backend as K

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import cv2
import skimage as sk


import os
import pickle
import numpy as np
import skimage as sk
import matplotlib.pyplot as plt
import tensorflow.keras.backend as K
from math import isnan
from tqdm import tqdm_notebook
from helper_functions import *

In [53]:
import random
c = np.random.randint(10, size=50)

In [ ]:
PATH = 'adata/'
PATH_MODEL = 'output/model/'
PATH_SAVE = 'output/save/'
SIZE        = (128,128,3)                           # taille des images en entrée
VALID_SPLIT  = 0.3    
LOAD_NET    = False                                 # Chargement du réseau
NET_NAME    = 'whaleFaceNet'                        # Network saved name
NBOF_EPOCHS = 250                                     # Number of epoch to train the network
START_EPOCH = 0
STEPS_PER_EPOCH = 150                                # Number of steps per epoch
VALIDATION_STEPS = 30   

On aura préalablement "augmenté" le jeu de donnés de manière à avoir au moins 2 photos par individus



In [ ]:
assert os.path.isdir(PATH), '[Error] Provided PATH for dataset does not exist.'

print('Chargement dataset...')

filenames = np.empty(0)
labels = np.empty(0)
idx = 0
for root,dirs,files in os.walk(PATH):
    if len(files)>1:
        for i in range(len(files)):
            files[i] = root + '/' + files[i]
        filenames = np.append(filenames,files)
        labels = np.append(labels,np.ones(len(files))*idx)
        idx += 1
assert len(labels)!=0, '[Error] No data provided.'

print('Done.')

print("Total nombres d'images importés: {:d}".format(len(labels)))

nbof_classes = len(np.unique(labels))
print('Total nombre de classes: {:d}'.format(nbof_classes))

Chargement dataset...
Done.
Total nombres d'images importés: 4789
Total nombre de classes: 447


In [ ]:
nbof_test = int(VALID_SPLIT*nbof_classes)

keep_test = np.less(labels,nbof_test)
keep_train = np.logical_not(keep_test)

filenames_test = filenames[keep_test]
labels_test = labels[keep_test]

filenames_train = filenames[keep_train]
labels_train = labels[keep_train]

print("Nombre d'images d'entrainements: " , len(filenames_train))
print("Number de classes d'entrainements: " ,nbof_classes-nbof_test)
print("Number d'images de test: " ,len(filenames_test))
print("Number de classes de test: " ,nbof_test)

Nombre d'images d'entrainements:  3899
Number de classes d'entrainements:  313
Number d'images de test:  890
Number de classes de test:  134


### Définition fonction de perte et calcul accuracy


In [ ]:
alpha = 0.3
def triplet(y_true,y_pred):
    
    a = y_pred[0::3]
    p = y_pred[1::3]
    n = y_pred[2::3]
    
    ap = K.sum(K.square(a-p),-1)
    an = K.sum(K.square(a-n),-1)

    return K.sum(tf.nn.relu(ap - an + alpha))

def triplet_acc(y_true,y_pred):
    a = y_pred[0::3]
    p = y_pred[1::3]
    n = y_pred[2::3]
    
    ap = K.sum(K.square(a-p),-1)
    an = K.sum(K.square(a-n),-1)
    
    return K.less(ap+alpha,an)

### Définition du modèle


In [ ]:
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Add, GlobalAveragePooling2D, DepthwiseConv2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Lambda, BatchNormalization

"""
Model numero présenté dans le papier avec le dropout et sans les "bottlneck layers"
"""

print('Definition du modèle {:s} ...'.format(NET_NAME))

# 128 meilleure choix de taille d'embedding en général voir le papier de recherche
emb_size = 32

### définition de l'architecture du modèle" ###

inputs = Input(shape=SIZE)
x = Conv2D(16, (7, 7), (2, 2), use_bias=False, activation='relu', padding='same')(inputs)
x = BatchNormalization()(x)
x = MaxPooling2D((3,3))(x)

for layer in [16,32,64,128,512]:
  x = Conv2D(layer, (3, 3), strides=(2,2), use_bias=False, activation='relu', padding='same')(x)
  r = BatchNormalization()(x)

  x = Conv2D(layer, (3, 3), use_bias=False, activation='relu', padding='same')(r)
  x = BatchNormalization()(x)
  r = Add()([r,x])

  x = Conv2D(layer, (3, 3), use_bias=False, activation='relu', padding='same')(r)
  x = BatchNormalization()(x)
  x = Add()([r,x])
    
x = GlobalAveragePooling2D()(x)
x = Flatten()(x)
x = Dropout(0.5)(x)
x = Dense(emb_size, use_bias=False)(x)
outputs = Lambda(lambda x: tf.nn.l2_normalize(x,axis=-1))(x)

model = tf.keras.Model(inputs,outputs)


### compilation model ###

model.compile(loss=triplet,
            optimizer='adam',
            metrics=[triplet_acc])

print('Done.')

#print(model.summary())

from contextlib import redirect_stdout

# sauvegarde de l'architecture du réseau dans un fichier texte
with open('modelsummary.txt', 'w') as f:
  with redirect_stdout(f):
    model.summary()

Definition du modèle whaleFaceNet ...
Done.


###Helper Functions
le preprocessing comme le resize normalisation sont effectués au sein de la fonction load_image

In [ ]:
from skimage.transform import resize

def define_adaptive_hard_triplets_batch(filenames,labels,predict,nbof_triplet=21*3, use_neg=True, use_pos=True):
    """
    Genere des "hard triplets" pour la "offline selection" 
    On va considerer tout le dataset filenames (qui est un subset)
    La fonction va retourner les valeurs predites    
    Args:
        -images: images from which the triplets will be created
        -labels: labels des images
        -predict: embeddings predits pour les images par le model entrainé
        -alpha: seuil pour la fonction de perte
    Returns:
        -triplets
        -y_triplets: label des triplets
        -pred_triplets: embedding prédit pour le triplet
    """
    # Check if we have the right number of triplets
    assert nbof_triplet%3 == 0
    
    # on récupère l'indice des classes du subset de fichiers envoyé en paramètre
    _,idx_classes = np.unique(labels,return_index=True)
    classes = labels[np.sort(idx_classes)]
   
    triplets = []
    y_triplets = np.empty(nbof_triplet)
    pred_triplets = np.empty((nbof_triplet,predict.shape[-1]))
    for i in range(0,nbof_triplet,3):
        # Chooses the first class randomly
        keep = np.equal(labels,classes[np.random.randint(len(classes))])
        keep_filenames = filenames[keep]
        keep_labels = labels[keep]
        
        # Chooses the first image among this class randomly
        idx_image1 = np.random.randint(len(keep_labels))
        
        # Computes the distance between the chosen image and the rest of the class
        if use_pos:
            dist_class = np.sum(np.square(predict[keep]-predict[keep][idx_image1]),axis=-1)

            idx_image2 = np.argmax(dist_class)
        else:
            idx_image2 = np.random.randint(len(keep_labels))
            j = 0
            while idx_image1==idx_image2:
                idx_image2 = np.random.randint(len(keep_labels))
                # Just to prevent endless loop:
                j += 1
                if j == 1000:
                    print("[Error: define_hard_triplets_batch] Endless loop.")
                    break
        
        triplets += [keep_filenames[idx_image1]]
        y_triplets[i] = keep_labels[idx_image1]
        pred_triplets[i] = predict[keep][idx_image1]
        triplets += [keep_filenames[idx_image2]]
        y_triplets[i+1] = keep_labels[idx_image2]
        pred_triplets[i+1] = predict[keep][idx_image2]
        
        # Computes the distance between the chosen image and the rest of the other classes
        not_keep = np.logical_not(keep)
        
        if use_neg:
            dist_other = np.sum(np.square(predict[not_keep]-predict[keep][idx_image1]),axis=-1)
            idx_image3 = np.argmin(dist_other) 
        else:
            idx_image3 = np.random.randint(len(filenames[not_keep]))
            
        triplets += [filenames[not_keep][idx_image3]]
        y_triplets[i+2] = labels[not_keep][idx_image3]
        pred_triplets[i+2] = predict[not_keep][idx_image3]

    return np.array(triplets), y_triplets, pred_triplets



def define_triplets_batch(filenames,labels,nbof_triplet = 21 * 3):
    """
    Generates offline soft triplet.
    Given a list of file names of pictures, their specific label and
    a number of triplet images, returns an array of triplet of images
    and their specific labels.

    Args:
     - filenames: array of strings. List of file names of the pictures. 
     - labels: array of integers.
     - nbof_triplet: integer. Has to be a multiple of 3.
     
     Returns:
     - triplet_train: array of pictures --> a 4D array. 
     - y_triplet: array of integers of same dimension as the first
     dimension of triplet_train. Contains the labels of the pictures.
    """
    triplet_train = []
    y_triplet = np.empty(nbof_triplet)
    classes = np.unique(labels)
    for i in range(0,nbof_triplet,3):
        # Pick a class and chose two pictures from this class
        classAP = classes[np.random.randint(len(classes))]
        keep = np.equal(labels,classAP)
        keep_classAP = filenames[keep]
        keep_classAP_idx = labels[keep]
        idx_image1 = np.random.randint(len(keep_classAP))
        idx_image2 = np.random.randint(len(keep_classAP))
        while idx_image1 == idx_image2:
            idx_image2 = np.random.randint(len(keep_classAP))

        triplet_train += [keep_classAP[idx_image1]]
        triplet_train += [keep_classAP[idx_image2]]
        y_triplet[i] = keep_classAP_idx[idx_image1]
        y_triplet[i+1] = keep_classAP_idx[idx_image2]
        # Pick a class for the negative picture
        classN = classes[np.random.randint(len(classes))]
        while classN==classAP:
            classN = classes[np.random.randint(len(classes))]
        keep = np.equal(labels,classN)
        keep_classN = filenames[keep]
        keep_classN_idx = labels[keep]
        idx_image3 = np.random.randint(len(keep_classN))
        triplet_train += [keep_classN[idx_image3]]
        y_triplet[i+2] = keep_classN_idx[idx_image3]
        
    return triplet_train, y_triplet



def image_generator(filenames, labels, batch_size=63):
    """
    Training generator for soft triplets.
    """
    while True:
        f_triplet, y_triplet = define_triplets_batch(filenames, labels, batch_size)
        i_triplet = load_images(f_triplet)
        yield (i_triplet, y_triplet)



def load_images(filenames):
    """
    Recupère les images à partir du nom de fichier la converty en np array et la normalise.
    """
    h,w,c = SIZE
    
    images = np.empty((len(filenames),h,w,c))

    for i,f in enumerate(filenames):
        img = sk.io.imread(f)/255.0
        img = resize(img, (h,w),anti_aliasing=True)
        images[i] = img/255.0
    return images



def predict_generator(filenames, batch_size=32):
    """
    Prediction generator.
    """
    for i in range(0,len(filenames),batch_size):
        images_batch = load_images(filenames[i:i+batch_size])
        yield images_batch



def online_adaptive_hard_image_generator(
    filenames,                  # Absolute path of the images
    labels,                     # Labels des images
    model,                      # A keras model
    loss,                       # Current loss of the model
    batch_size      =63,        # Batch size (has to be a multiple of 3 for WhaleFaceNet)
    nbof_subclasses =10):       # Nombres de sous classes ou les triples vont être selectionné
    """
    Generator to select online hard triplets for training.
    Include an adaptive control on the number of hard triplets included during the training.
    """
    
    hard_triplet_ratio = 0
    nbof_hard_triplets = 0
    while True:
        # Selection d'un certain de nombres de subclasses
        classes = np.unique(labels)
        """
        Pour limiter la nombre de calcul lors de la predition on ne va pas 
        calculer nbof_subclasses predictions pour la generation des hard triplets
        mais calculer un nombre directement impacté par le valeur de l'accuracy.
        Plus l'accuracy est elevé plus on va chercher à generer de hard triplets 
        Ainsi on va en génerer int(nbof_subclasses*hard_triplet_ratio)+2.
        """
        # on récupère les sous classes aléatoirement on en choissisant un nombre linéairement lié au hard_triplet_ratio 
        subclasses = np.random.choice(classes,size=int(nbof_subclasses*hard_triplet_ratio)+2,replace=False)
        # keep_classes est un masque sur les labels à garder
        keep_classes = np.equal(labels,subclasses[0])
        for i in range(1,len(subclasses)):
            keep_classes = np.logical_or(keep_classes,np.equal(labels,subclasses[i]))

        subfilenames = filenames[keep_classes]
        sublabels = labels[keep_classes]

        # prédictions sur les fichiers et labels à conserver
        predict = model.predict(predict_generator(subfilenames, 32),
                                          steps=int(np.ceil(len(subfilenames)/32)))

        # 
        f_triplet_hard, y_triplet_hard, predict_hard = define_adaptive_hard_triplets_batch(subfilenames, sublabels, predict, nbof_hard_triplets*3, use_neg=True, use_pos=True)
        f_triplet_soft, y_triplet_soft, predict_soft = define_adaptive_hard_triplets_batch(subfilenames, sublabels, predict, batch_size-nbof_hard_triplets*3, use_neg=False, use_pos=False)

        f_triplet = np.append(f_triplet_hard,f_triplet_soft)
        y_triplet = np.append(y_triplet_hard,y_triplet_soft)

        predict = np.append(predict_hard, predict_soft, axis=0)
        
        # Proportion de hard triplets dans le batch generé
        # hard_triplet_ratio = max(0,1.2/(1+np.exp(-10*acc+5.3))-0.19)
        hard_triplet_ratio = np.exp(-loss * 10 / batch_size)

        if isnan(hard_triplet_ratio):
            hard_triplet_ratio = 0
        nbof_hard_triplets = int(batch_size//3 * hard_triplet_ratio)
        
        i_triplet = load_images(f_triplet)
        
        yield (i_triplet, y_triplet)

In [ ]:
from skimage import io

# Model training.
histories = []
crt_loss = 0.6
crt_acc = 0
batch_size = 3*10
nbof_subclasses = 40

# Create saving folders
if not os.path.isdir(PATH_MODEL):
    os.makedirs(PATH_MODEL)
if not os.path.isdir(PATH_SAVE):
    os.makedirs(PATH_SAVE)

# Bug fixed: keras models are to be initialized by a training on a single batch
for images_batch,labels_batch in online_adaptive_hard_image_generator(
    filenames_train,
    labels_train,
    model,
    crt_acc,
    batch_size = batch_size,
    nbof_subclasses=nbof_subclasses):
    h = model.train_on_batch(images_batch,labels_batch)
    break

if START_EPOCH > 0:
  print('Loading model from {:s}{:s}.{:d}.h5 ...'.format(PATH_MODEL,NET_NAME,START_EPOCH))

  model = tf.keras.models.load_model(
      '{:s}{:s}.{:d}.h5'.format(PATH_MODEL,NET_NAME,START_EPOCH),
      custom_objects={'triplet':triplet,'triplet_acc':triplet_acc})

  print('Done.')

for i in range(START_EPOCH,START_EPOCH+NBOF_EPOCHS):
    print("Beginning epoch number: "+str(i))

    hard_triplet_ratio = np.exp(-crt_loss * 10 / batch_size)
    nbof_hard_triplets = int(batch_size//3 * hard_triplet_ratio)
    
    print("Current hard triplet ratio: " + str(hard_triplet_ratio))
    
    histories += [model.fit(
        online_adaptive_hard_image_generator(filenames_train,labels_train,model,crt_loss,batch_size,nbof_subclasses=nbof_subclasses),
        steps_per_epoch=STEPS_PER_EPOCH,
        epochs=1,
        validation_data=image_generator(filenames_test,labels_test,batch_size),
        validation_steps=VALIDATION_STEPS)]
    
    crt_loss = histories[-1].history['loss'][0]
    crt_acc = histories[-1].history['triplet_acc'][0]

    # Save model
    model.save('{:s}{:s}.{:d}.h5'.format(PATH_MODEL,NET_NAME,i))
    
    # Save history
    loss = np.empty(0)
    val_loss = np.empty(0)
    acc = np.empty(0)
    val_acc = np.empty(0)

    for history in histories:
        loss = np.append(loss,history.history['loss'])
        val_loss = np.append(val_loss,history.history['val_loss'])
        acc = np.append(acc,history.history['triplet_acc'])
        val_acc = np.append(val_acc,history.history['val_triplet_acc'])

    history_ = np.array([loss,val_loss,acc,val_acc])
    np.save('{:s}{:s}.{:d}.npy'.format(PATH_SAVE,NET_NAME,i),history_)

Loading model from output/model/whaleFaceNet.9.h5 ...
Done.
Beginning epoch number: 9
Current hard triplet ratio: 0.8187307530779818
150/150 [==============================] - 4007s 27s/step - loss: 4.3652 - triplet_acc: 0.1927 - val_loss: 2.4805 - val_triplet_acc: 0.0267
Beginning epoch number: 10
Current hard triplet ratio: 0.2333855058876403
 41/150 [=======>......................] - ETA: 14:20 - loss: 3.8794 - triplet_acc: 0.2268

### Evaluation
On génère plein de paires fausses et vraies et l'on regarde si notre modèle les classe comme appartenant au même individus ou pas (Le kernel peut être redemarrer à partir d'ici pour liberer la mémoire)

In [ ]:
START_EPOCH = 10
print('Loading model from {:s}{:s}.{:d}.h5 ...'.format(PATH_MODEL,NET_NAME,START_EPOCH))

model = tf.keras.models.load_model(
    '{:s}{:s}.{:d}.h5'.format(PATH_MODEL,NET_NAME,START_EPOCH),
    custom_objects={'triplet':triplet,'triplet_acc':triplet_acc})

print('Done.')

In [ ]:
print('Verification task, pairs creation...')

# nombre arbitraire mais doit être assez gros pour une valeur proche de la réalité
NBOF_PAIRS = 15000
#NBOF_PAIRS = len(images_test)

# Create pairs
h,w,c = SIZE
pairs = []
issame = np.empty(NBOF_PAIRS)
class_test = np.unique(labels_test)
for i in range(NBOF_PAIRS):
    alea = np.random.rand()
    # Pair of different dogs
    if alea < 0.5:
        # Choose classes:
        class1 = np.random.randint(len(class_test))
        class2 = np.random.randint(len(class_test))
        while class1==class2:
            class2 = np.random.randint(len(class_test))
            
        # Extract images of this class:
        images_class1 = filenames_test[np.equal(labels_test,class1)]
        images_class2 = filenames_test[np.equal(labels_test,class2)]
        
        # Chose an image amoung these selected images
        pairs = pairs + [images_class1[np.random.randint(len(images_class1))]]
        pairs = pairs + [images_class2[np.random.randint(len(images_class2))]]
        issame[i] = 0
    # Pair of same dogs
    else:
        # Choose a class
        clas = np.random.randint(len(class_test))
        images_class = filenames_test[np.equal(labels_test,clas)]
        
        # Select two images from this class
        idx_image1 = np.random.randint(len(images_class))
        idx_image2 = np.random.randint(len(images_class))
        while idx_image1 == idx_image2:
            idx_image2 = np.random.randint(len(images_class))
        
        pairs = pairs + [images_class[idx_image1]]
        pairs = pairs + [images_class[idx_image2]]
        issame[i] = 1

print('Done.')

In [ ]:
print('Verification task, model evaluation...')

predict=model.predict(predict_generator(pairs, batch_size = 3), steps=np.ceil(len(pairs)/batch_size))
# # Separates the pairs
emb1 = predict[0::2]
emb2 = predict[1::2]

# # Computes distance between pairs
diff = np.square(emb1-emb2)
dist = np.sum(diff,1)
dist.shape

In [ ]:
best = 0
best_t = 0
thresholds = np.arange(0.001,4,0.001)
for i in range(len(thresholds)):
    less = np.less(dist, thresholds[i])
    acc = np.logical_not(np.logical_xor(less, issame))
    acc = acc.astype(float)
    out = np.sum(acc)
    out = out/len(acc)
    if out > best:
        best_t = thresholds[i]
        best = out


print('Done.')
print("Best threshold: " + str(best_t))
print("Best accuracy: " + str(best))

In [ ]:
vraieP = len(issame[issame == 1])
fausseP = len(issame[issame == 0])

# Test: Look at wrong pairs
t = best_t
fpositifs = []
fnegatifs = []
vpositifs = []
vnegatifs = []
for i in range(len(dist)):
    # faux positifs
    if issame[i] == 0 and dist[i]<t:
        fpositifs += [i]
    # false negatifs
    if issame[i] == 1 and dist[i]>t:
        fnegatifs += [i]
    # vraie positifs
    if issame[i] == 1 and dist[i]<t:
        vpositifs += [i]
    # vraie negatifs
    if issame[i] == 0 and dist[i]>t:
        vnegatifs += [i]

conf_mat = [[len(vpositifs)/vraieP,len(fnegatifs)/vraieP],[len(fpositifs)/fausseP,len(vnegatifs)/fausseP]]
print("conf_matrix")
print(conf_mat)

###Détection
Première approches simple mais gloutonne sur la detection

In [ ]:
import re 
filenames,labels = [],[]
for root,dirs,files in os.walk(PATH):
    for file in files:  
        file = root + "/" + file
        dir = re.findall("(?<=adata\/)(.*?)(?=\/)",file).pop()
        filenames.append(file)
        labels.append(dir)

In [ ]:
photos = 'adata/17/17.0.jpg'
idx = filenames.index(photos)

labels_bis = np.delete(labels,idx)
filenames_bis = np.delete(filenames,idx)

pairs = []
for file in filenames_bis:
    pairs.append(photos)
    pairs.append(file)


predict = model.predict(predict_generator(pairs, batch_size = 32))
predict.shape

# # # Separates the pairs
emb1 = predict[0::2]
emb2 = predict[1::2]

# # # Computes distance between pairs
diff = np.square(emb1-emb2)
dist = np.sum(diff,1)

labels_ = dict()
labels_[labels[0]] = [0]

tmp_classe = ""
for i in range(1,len(labels)):
    if labels[i] != labels[i-1] :
        tmp_classe = labels[i-1]
        labels_[tmp_classe].append(i-1)
        tmp_classe = labels[i]
        labels_[tmp_classe] = [i]
        
labels_[tmp_classe].append(len(labels)-1)

dist_mean = dict()

classe = np.unique(labels)
for classe_name in classe:
    x,y = labels_[classe_name]
    dist_mean[classe_name] = dist[x:y].mean()

print(max(dist_mean, key=dist_mean.get))

A suivre 
